In [1]:
from langchain.prompts import ChatPromptTemplate
from langchain_ollama import ChatOllama
from langchain.schema.output_parser import StrOutputParser

In [3]:
prompt = ChatPromptTemplate.from_template(
    "Tell me a joke about {topic}"
)
llm = ChatOllama(model="llama3.2:latest", temperature=0.5)
output_parser = StrOutputParser()

Simple Chain

In [4]:
chain = prompt | llm | output_parser

In [6]:
chain.invoke({"topic": "chickens"})

'Why did the chicken go to the doctor?\n\nBecause it had fowl breath! (get it?)'

## More complex chain
Add a runnable map

In [7]:
from langchain_ollama.embeddings import OllamaEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch

In [9]:
vector_store = DocArrayInMemorySearch.from_texts(
    ["harrison worked at kensho", "bears like to eat honey"],
    embedding=OllamaEmbeddings(model="mxbai-embed-large")
    )
retriever = vector_store.as_retriever()

In [11]:
retriever.get_relevant_documents("where did harrison work?")

/var/folders/0t/xlg3nq8x63j3nxrs9x50yw9m0000gn/T/ipykernel_21519/3310280720.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retriever.get_relevant_documents("where did harrison work?")


[Document(metadata={}, page_content='harrison worked at kensho'),
 Document(metadata={}, page_content='bears like to eat honey')]

In [12]:
retriever.get_relevant_documents("what do bears like to eat")

[Document(metadata={}, page_content='bears like to eat honey'),
 Document(metadata={}, page_content='harrison worked at kensho')]

In [13]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [14]:
from langchain.schema.runnable import RunnableMap

In [15]:
chain = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
}) | prompt | llm | output_parser

In [16]:
chain.invoke({"question": "where did harrison work?"})

'Harrison worked at Kensho.'

In [17]:
chain.invoke({"question": "What is Harisson known for?"})

'Based on the provided context, it appears that Harrison is associated with "Kensho". However, there is no additional information about what Kensho is or what Harrison\'s role or expertise in that field might be. Therefore, I can only infer that Harrison might be known for working at Kensho, but this is not a definitive answer.'

In [18]:
inputs = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
})

In [22]:
ins = inputs.invoke({"question": "where did harrison work?"})
ins

{'context': [Document(metadata={}, page_content='harrison worked at kensho'),
  Document(metadata={}, page_content='bears like to eat honey')],
 'question': 'where did harrison work?'}

In [23]:
pr = prompt.invoke(ins)
pr

ChatPromptValue(messages=[HumanMessage(content="Answer the question based only on the following context:\n[Document(metadata={}, page_content='harrison worked at kensho'), Document(metadata={}, page_content='bears like to eat honey')]\n\nQuestion: where did harrison work?\n", additional_kwargs={}, response_metadata={})])

In [26]:
res = llm.invoke(pr)
res.content

'Harrison worked at Kensho.'

In [29]:
parsed = output_parser.invoke(res)
parsed

'Harrison worked at Kensho.'

In [34]:
from langchain_core.utils.function_calling import convert_to_openai_tool
from langchain_core.tools import Tool

Bindings

In [47]:
# functions = [
#     {
#       "name": "weather_search",
#       "description": "Search for weather given an airport code",
#       "parameters": {
#         "type": "object",
#         "properties": {
#           "airport_code": {
#             "type": "string",
#             "description": "The airport code to get the weather for"
#           },
#         },
#         "required": ["airport_code"]
#       }
#     }
#   ]
def weather_search(airport_code: str) -> str:
    """Actual implementation of weather search"""
    # Replace this with real API call
    return f"Weather at {airport_code}: 25°C, sunny with light winds"

In [48]:
weather_tool = Tool(
    name="weather_search",
    description="Search for weather given an airport code",
    func=weather_search,
    args_schema={
        "airport_code": {
            "type": "string",
            "description": "The airport code to get the weather for"
        }
    }
)

# Convert to OpenAI-compatible format
functions = [convert_to_openai_tool(weather_tool)]

In [49]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful travel assistant. Use tools when needed."),
    ("human", "{input}")
])
model = ChatOllama(model="llama3.2", temperature=0).bind_tools([weather_tool])

In [54]:
runnable = prompt | model 

In [55]:
runnable.invoke({"input": "what is the weather in sf"})

AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2025-03-13T11:55:39.147944Z', 'done': True, 'done_reason': 'stop', 'total_duration': 502508333, 'load_duration': 32320833, 'prompt_eval_count': 162, 'prompt_eval_duration': 118000000, 'eval_count': 18, 'eval_duration': 351000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-deabef58-d52e-4b00-9131-54f939724dce-0', tool_calls=[{'name': 'weather_search', 'args': {'code': 'SFO'}, 'id': '1ec18c0c-3235-40b9-9a6f-07e08c37ea86', 'type': 'tool_call'}], usage_metadata={'input_tokens': 162, 'output_tokens': 18, 'total_tokens': 180})